In [1]:
import os
import openai
from dotenv import load_dotenv
import json

# Load environment variables from the '~/.my_env_vars' file
env_file = os.path.expanduser("~/.my_env_vars")
load_dotenv(env_file)

# Load your API key from an environment variable or secret management service
openai.api_key = os.getenv("OPENAI_API_KEY")

prompt = "Please rewrite the following complex sentence in order to make it easier to understand by non-native speakers of English. You can do so by replacing complex words with simpler synonyms (i.e. paraphrasing), deleting unimportant information (i.e. compression), and/or splitting a long complex sentence into several simpler ones. The final simplified sentence needs to be grammatical, fluent, and retain the main ideas of its original counterpart without altering its meaning.\n\n%s"

In [2]:
# read ../yao/new_wiki_original_sentences_4.txt
batch_num = 2
with open(f'../data/yao/new_wiki_original_sentences_{batch_num}.txt', 'r') as f:
    lines = f.readlines()
    # remove \n
    lines = [x.strip() for x in lines]

In [3]:
print(len(lines))

30


In [4]:
def callGPT(prompt):
    response = openai.Completion.create(
        engine="text-davinci-003", 
        prompt=prompt, 
        temperature=1,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        max_tokens=512)
    responses = [x['text'] for x in response['choices']]
    if len(responses) > 1:
        print(f"More than one response! {responses}")
    responses = responses[0]
    responses = responses.strip()
    return responses

def callChatGPT(prompt, model_name):
    response = openai.ChatCompletion.create(
        model=model_name, 
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=1,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        max_tokens=512)
    response = response["choices"][0]["message"]["content"]
    return response

In [5]:
# output_davinci = []
# for sent in lines:
#     # print(sent)
#     response = callGPT(prompt % sent)
#     output_davinci.append(response)
# # save to file gpt-outputs/zero-shot-003-batch-4.json
# with open(f'gpt-outputs/zero-shot-003-batch-{batch_num}.json', 'w') as f:
#     json.dump(output_davinci, f, indent=4)

In [6]:
# current turbo is gpt-3.5-turbo-0301, gpt-4 is gpt-4-0314
for model in ["gpt-4"]: # "gpt-3.5-turbo", 
    output_chat = []
    for sent in lines:
        response = callChatGPT(prompt % sent, model_name=model)
        output_chat.append(response)
    # save to file
    if model == "gpt-3.5-turbo":
        with open(f'gpt-outputs/zero-shot-turbo-batch-{batch_num}.json', 'w') as f:
            json.dump(output_chat, f, indent=4)
    else:
        with open(f'gpt-outputs/zero-shot-gpt-4-batch-{batch_num}.json', 'w') as f:
            json.dump(output_chat, f, indent=4)

In [7]:
output_chat

['The club posted on social media that customers stopped the gunman. They said they are very sad about the pointless attack on their community. They also offered sympathy to the victims and their families.',
 'The non-profit group Iran Human Rights says that by November 19, 2022, at least 378 people, including 43 minors, had been killed by the government during protests. The government used tear gas and guns, making these protests the deadliest since 2019-2020 when more than 1,500 people died.',
 "The movie features Julie Ledru as Julia, a young woman in public housing who doesn't get along with her mom. She loves motocross and tries to find a chance to join the sport by pretending to be a buyer on shopping websites so she can test ride bikes.",
 "The movie is about a rich, private family living alone in their big house to hide their daughter Hélène's pregnancy from a short affair. Their secret life is disrupted when Hélène leaves to find the baby's father after giving birth.",
 'Befor